In [1]:
# Use a different environment
# Instead of opencv-python, use opencv-python-headless (requiring a different environment)

# https://github.com/edavalosanaya/plot3d
import plot3d

# In a separate terminal, run to start the server:
# plot3d

# Imports
import cv2
import time
import pathlib
import os
from tqdm import tqdm
import numpy as np
import imutils
import trimesh
from scipy.spatial.transform import Rotation as R
import pandas as pd

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent.parent
DATA_DIR = GIT_ROOT / "data" / 'AIED2024'

# Append ZoeDepth to path
import sys
sys.path.append('ZoeDepth')

/tmp/ipykernel_3112533/691582857.py:20: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [62]:
def get_intrinsics(H,W):
    """
    Intrinsics for a pinhole camera model.
    Assume fov of 55 degrees and central principal point.
    """
    f = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    cx = 0.5 * W
    cy = 0.5 * H
    return np.array([[f, 0, cx],
                     [0, f, cy],
                     [0, 0, 1]])

def depth_to_points(depth, R=None, t=None):

    K = get_intrinsics(depth.shape[1], depth.shape[2])
    Kinv = np.linalg.inv(K)
    if R is None:
        R = np.eye(3)
    if t is None:
        t = np.zeros(3)

    # M converts from your coordinate to PyTorch3D's coordinate system
    M = np.eye(3)
    M[0, 0] = -1.0
    M[1, 1] = -1.0

    height, width = depth.shape[1:3]

    x = np.arange(width)
    y = np.arange(height)
    coord = np.stack(np.meshgrid(x, y), -1)
    coord = np.concatenate((coord, np.ones_like(coord)[:, :, [0]]), -1)  # z=1
    coord = coord.astype(np.float32)
    # coord = torch.as_tensor(coord, dtype=torch.float32, device=device)
    coord = coord[None]  # bs, h, w, 3

    D = depth[:, :, :, None, None]
    # print(D.shape, Kinv[None, None, None, ...].shape, coord[:, :, :, :, None].shape )
    pts3D_1 = D * Kinv[None, None, None, ...] @ coord[:, :, :, :, None]
    # pts3D_1 live in your coordinate system. Convert them to Py3D's
    pts3D_1 = M[None, None, None, ...] @ pts3D_1
    # from reference to targe tviewpoint
    pts3D_2 = R[None, None, None, ...] @ pts3D_1 + t[None, None, None, :, None]
    # pts3D_2 = pts3D_1
    # depth_2 = pts3D_2[:, :, :, 2, :]  # b,1,h,w
    return pts3D_2[:, :, :, :3, 0][0]

def depth_edges_mask(depth):
    """Returns a mask of edges in the depth map.
    Args:
    depth: 2D numpy array of shape (H, W) with dtype float32.
    Returns:
    mask: 2D numpy array of shape (H, W) with dtype bool.
    """
    # Compute the x and y gradients of the depth map.
    depth_dx, depth_dy = np.gradient(depth)
    # Compute the gradient magnitude.
    depth_grad = np.sqrt(depth_dx ** 2 + depth_dy ** 2)
    # Compute the edge mask.
    mask = depth_grad > 0.05
    return mask

def create_triangles(h, w, mask=None):
    """Creates mesh triangle indices from a given pixel grid size.
        This function is not and need not be differentiable as triangle indices are
        fixed.
    Args:
    h: (int) denoting the height of the image.
    w: (int) denoting the width of the image.
    Returns:
    triangles: 2D numpy array of indices (int) with shape (2(W-1)(H-1) x 3)
    """
    x, y = np.meshgrid(range(w - 1), range(h - 1))
    tl = y * w + x
    tr = y * w + x + 1
    bl = (y + 1) * w + x
    br = (y + 1) * w + x + 1
    triangles = np.array([tl, bl, tr, br, tr, bl])
    triangles = np.transpose(triangles, (1, 2, 0)).reshape(
        ((w - 1) * (h - 1) * 2, 3))
    if mask is not None:
        mask = mask.reshape(-1)
        triangles = triangles[mask[triangles].all(1)]
    return triangles

def get_mesh(image, depth, keep_edges=False):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pts3d = depth_to_points(depth[None])
    pts3d = pts3d.reshape(-1, 3)

    # Create a trimesh mesh from the points
    # Each pixel is connected to its 4 neighbors
    # colors are the RGB values of the image

    verts = pts3d.reshape(-1, 3)
    image = np.array(image)
    if keep_edges:
        triangles = create_triangles(image.shape[0], image.shape[1])
    else:
        triangles = create_triangles(image.shape[0], image.shape[1], mask=~depth_edges_mask(depth))
    colors = image.reshape(-1, 3)
    mesh = trimesh.Trimesh(vertices=verts, faces=triangles, vertex_colors=colors)

    # Save as glb
    return mesh

def compute_3D_point(x, y, Z, H, W):
    """
    Compute the 3D point in the camera coordinate system from an image coordinate and depth.

    Parameters:
    - x, y: The image coordinates (pixels)
    - Z: The depth value (distance along the camera's viewing axis)
    - f_x, f_y: The camera's focal lengths along the X and Y axes (pixels)
    - c_x, c_y: The optical center of the camera (pixels)

    Returns:
    A tuple (X, Y, Z) representing the 3D point in the camera coordinate system.
    """
    # 
    fy = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    fx = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    cx = 0.5 * W
    cy = 0.5 * H
    print(fx, fy, cx, cy)

    # Normalize the 2D coordinates
    x_prime = (x - cx) / fx
    y_prime = (y - cy) / fy

    # Apply the depth to get the 3D point
    X = x_prime * Z
    Y = y_prime * Z

    return np.array([X, Y, Z])


def draw_gaze(x, y, length, img, pitchyaw, thickness=2, color=(255, 255, 0),sclae=2.0):
    """Draw gaze angle on given image with a given eye positions."""
    pos = (int(x), int(y))
    if len(img.shape) == 2 or img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    dx = -length * np.sin(pitchyaw[0]) * np.cos(pitchyaw[1])
    dy = -length * np.sin(pitchyaw[1])
    cv2.arrowedLine(img, tuple(np.round(pos).astype(np.int32)),
                   tuple(np.round([pos[0] + dx, pos[1] + dy]).astype(int)), color,
                   thickness, cv2.LINE_AA, tipLength=0.18)
    return img

def create_arrow(shaft_length=1.0, shaft_radius=0.01, head_length=0.2, head_radius=0.1, thickness=1):
    """
    Create an arrow mesh.
    
    Parameters:
    - shaft_length: Length of the shaft.
    - shaft_radius: Radius of the shaft.
    - head_length: Length of the head.
    - head_radius: Radius of the head.
    
    Returns:
    - A trimesh object representing the arrow.
    """
    # Create the shaft of the arrow (cylinder)
    shaft = trimesh.creation.cylinder(radius=shaft_radius*thickness, height=shaft_length, sections=32)
    shaft.apply_translation((0, 0, shaft_length/2))
    
    # Create the head of the arrow (cone)
    head = trimesh.creation.cone(radius=head_radius*thickness, height=head_length*thickness, sections=32)
    head.apply_translation((0, 0, shaft_length))
    
    # Combine the shaft and the head
    arrow = trimesh.util.concatenate([shaft, head])

    arrow.visual.face_colors = [1, 0, 0, 0.5]
    arrow.visual.vertex_colors = [1, 0, 0, 0.5]
    
    return arrow


def create_oriented_arrow(origin, pitch, yaw, length=1.0, thickness=1):

    # Convert pitch, yaw, and roll to rotvec
    rotation = R.from_euler('xyz', [0, pitch, yaw])
    initial_vector = np.array([0,0,1])

    # Compute the endpoint based on origin, pitch, yaw, and length
    endpoint = rotation.apply(initial_vector)*length
    
    # Create an arrow mesh
    # arrow = trimesh.creation.arrow(radius=0.05, height=length)
    arrow = create_arrow(length, thickness=5)
    
    # Compute the direction vector for the arrow
    direction = endpoint - origin
    direction /= np.linalg.norm(direction) # Normalize the direction vector
    
    # Compute the rotation needed to align the arrow with the direction vector
    # Default arrow direction is along the z-axis (0, 0, 1)
    # default_direction = np.array([0, 0, 1])
    # rotation_vector = np.cross(default_direction, direction)
    # rotation_angle = np.arccos(np.dot(default_direction, direction))
    # rotation = R.from_rotvec(rotation_vector * rotation_angle)
    
    # Apply rotation to the arrow
    # arrow.apply_transform(rotation.as_matrix())
    rt = np.eye(4)
    rt[:3,:3] = rotation.as_matrix()
    rt[:3,-1] = origin
    arrow.apply_transform(rt)
    
    return arrow

# Create a plot
plot = plot3d.Plot(port=9000)

# Reset the 3D Plot
plot.reset()

DISPLAY_RATIO = 80
display = trimesh.creation.box(extents=np.array([0.6, 0.25, 0.01])*DISPLAY_RATIO)
display.visual.face_colors = [0, 1, 0, 0.5]
display.visual.vertex_colors = [0, 1, 0, 0.5]

# Add the monitor rectangle
# X -> blue, Y -> green, Z -> yellow
r = R.from_euler('xyz', np.radians(np.array([30,110,30])))
t = np.array([-3, 0, -9])*4
rt = np.eye(4)
rt[:3, :3] = r.as_matrix()
rt[:3, 3] = t

display.apply_transform(rt)
plot.add_mesh(f'display', display)

FLOOR_RATIO = 70
floor = trimesh.creation.box(extents=np.array([2, 2, 0.01])*FLOOR_RATIO)
floor.visual.face_colors = [0, 0, 1, 0.5]
floor.visual.vertex_colors = [0, 0, 1, 0.5]

# Add the floor rectangle
r = R.from_euler('xyz', np.radians(np.array([82,0,0])))
t = np.array([0, 0, -22])*4
rt = np.eye(4)
rt[:3, :3] = r.as_matrix()
rt[:3, 3] = t
floor.apply_transform(rt)

plot.add_mesh("floor", floor)

sphere = trimesh.creation.uv_sphere(radius=0.25)
sphere.visual.face_colors = [0, 0, 1, 0.5]
sphere.visual.vertex_colors = [0, 0, 1, 0.5]

def process(tracking_file: pathlib.Path, vid_file: pathlib.Path, depth_file: pathlib.Path, gaze_file: pathlib.Path):
    assert tracking_file.exists()
    assert vid_file.exists()
    assert depth_file.exists()
    assert gaze_file.exists()

    # Load the gaze vectors and the corresponding CSV with BBox info
    faces_df = pd.read_csv(tracking_file)
    gaze_df = pd.read_csv(gaze_file)

    # Load the RGB and depth videos
    cap = cv2.VideoCapture(str(vid_file))
    LENGTH = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    depth_cap = cv2.VideoCapture(str(depth_file))

    # Memory containers
    last_info = []

    for i in tqdm(range(LENGTH), total=LENGTH):

        # Load frame
        r_ret, rgb = cap.read()
        d_ret, depth = depth_cap.read()

        if not r_ret or not d_ret:
            break

        depth = cv2.cvtColor(depth, cv2.COLOR_BGR2GRAY)

        # Get the gaze vector
        faces = faces_df[faces_df['Frame'] == i]
        gaze_vectors = gaze_df[gaze_df['frame'] == i]

        j = 0
        for (_, face) in faces.iterrows():
            # Get the gaze vector
            # print(gaze_vectors)
            gaze_vector = gaze_vectors[gaze_vectors['tracked_id'] == face["Student_ID"]].iloc[0]
            pitch = gaze_vector['pitch']
            yaw = gaze_vector['yaw']

            # Compute the centroid of the face
            centroid = (face.X + face.Width/2, face.Y + face.Height/2)
            centroid_depth = depth[int(centroid[1]), int(centroid[0])]
            face_t = compute_3D_point(centroid[0], centroid[1], centroid_depth, depth.shape[0], depth.shape[1])
            face_t[-1] = face_t[-1]*-1
            face_t[0] = face_t[0]*-1
            line_start = face_t.copy()

            # 2D Data
            # Draw in the 2D image
            cv2.circle(rgb, (int(centroid[0]), int(centroid[1])), 5, (0, 255, 0), -1)

            # Draw the gaze vector
            draw_gaze(centroid[0], centroid[1], 100, rgb, [pitch, yaw], color=(0,255,0))

            # 3D Data
            # Make copy of spher and apply transform
            sphere_copy = sphere.copy()
            sphere_copy.apply_transform(trimesh.transformations.translation_matrix(face_t))

            # Draw spheres in the 3D plot
            if f"face-{j}" in plot.client.visuals:
                plot.update_mesh(f'face-{j}', sphere_copy)
            else:
                plot.add_mesh(f'face-{j}', sphere_copy)

            arrow = create_oriented_arrow(line_start, pitch, yaw, length=5, thickness=15)

            if f"gaze-{j}" in plot.client.visuals:
                plot.update_mesh(f"gaze-{j}", arrow)
            else:
                plot.add_mesh(f"gaze-{j}", arrow)

            j += 1
                
        # Resize
        sm_rgb = imutils.resize(rgb, width=500)
        sm_depth = imutils.resize(depth, width=500)

        mesh = get_mesh(sm_rgb, sm_depth, keep_edges=True)
        mesh.apply_transform(trimesh.transformations.rotation_matrix(np.pi, [1,0,0]))

        # Plot the frame
        plot.plot_image(sm_rgb)
        if i == 0:
            plot.add_mesh('mesh', mesh)
        else:
            plot.update_mesh('mesh', mesh)

        # Save to PLY
        # 400 camera distance
        # filepath = DATA_DIR / 'mesh' / 'g1d1.ply'
        # mesh.apply_translation(-mesh.centroid)
        # mesh.export(str(filepath), file_type='ply')
        break
        # time.sleep(1)


try:
    process(
        DATA_DIR / 'trackings' / 'Day1Group1Camera2_with_student_IDs.csv',
        DATA_DIR / "videos" / "day1" / "block-a-blue-day1-first-group-cam2.mp4",
        DATA_DIR / 'depths' / 'day1' / "group1-depth-cam2.mp4",
        DATA_DIR / 'gaze_vectors' / "gaze_vector_d1g1.csv",
    )
except KeyboardInterrupt:
    # plot.reset()
    pass

  0%|          | 0/13464 [00:00<?, ?it/s]

1844.1428418923194 1844.1428418923194 960.0 540.0
1844.1428418923194 1844.1428418923194 960.0 540.0
1844.1428418923194 1844.1428418923194 960.0 540.0
1844.1428418923194 1844.1428418923194 960.0 540.0
1844.1428418923194 1844.1428418923194 960.0 540.0
1844.1428418923194 1844.1428418923194 960.0 540.0


In [43]:
def describe_mesh(mesh):
    v = mesh.vertices
    print(v.min(axis=0))
    print(v.max(axis=0))
    print(v.mean(axis=0))

room_mesh = trimesh.load_mesh(DATA_DIR / 'mesh' / 'g1d1.ply')
describe_mesh(room_mesh)

[ -53.13694    -103.69874573  -28.41329575]
[ 53.36066818 123.30125427  32.7595787 ]
[ 14.25589973  58.17774504 -14.71972814]


In [40]:
SIZE=125
display = trimesh.creation.box(extents=np.array([0.5, 0.2, 0.01])*SIZE)
display.visual.face_colors = [1, 0, 0, 0.5]
display.visual.vertex_colors = [1, 0, 0, 0.5]

filepath = DATA_DIR / 'mesh' / 'display.ply'
display.apply_translation(-display.centroid)
display.export(str(filepath), file_type='ply')
describe_mesh(display)

surface = trimesh.creation.box(extents=np.array([2, 2, 0.01])*SIZE)
surface.visual.face_colors = [1, 0, 0, 0.5]
surface.visual.vertex_colors = [1, 0, 0, 0.5]

filepath = DATA_DIR / 'mesh' / 'surface.ply'
surface.apply_translation(-surface.centroid)
surface.export(str(filepath), file_type='ply')
describe_mesh(surface)

[-31.25  -12.5    -0.625]
[31.25  12.5    0.625]
[0. 0. 0.]
[-125.    -125.      -0.625]
[125.    125.      0.625]
[0. 0. 0.]
